## 🛠️ **ETL (Extract, Transform, Load)**
### **📂Procesamiento del 1er archivo: `steam_games.json.gz`**


📂 **steam_games.json**:

- 🗑️ Eliminamos columnas que no vamos a usar: publisher, title, url, early_access, reviews_url, specs.

🔄 `ID:`

- ~~Cambiamos su tipo de dato a str str o in=?~~
- Eliminamos nulos y repetidos

`release_date`:

- Extraigo el año de la fecha en la columna 'release_date'.
- Reemplazamos los datos sin fecha por -1.
- Cambiamos tipo de dato a int.

`price`:

- Asignmos valores correspondientes a precios con textos adicional.
- Rellenamos missing values buscando en tags si el juego es gratis.
- Rellena los valores nulos resultantes con -1.
- Convierte la columna 'price' a tipo float.

`genres:`

- Rellenamos la informacion faltante de genres, con los genres disponibles en la columna tags.
- 
- .

🔄 `app_name` en string

🔄 `tags` en string

🗃️ `Genres `: Rellenamos la informacion faltante de genres en cada juego, con los genres disponibles en la columna tag

#### 📦 **EXTRACT**

####  **Importamos las librerías que vamos a usar**


In [12]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet

import inspect

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
from data_utils import data_type_check, duplicados_columna
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### **Descomprimimos el archivos gz** 

In [7]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/steam_games.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)


Archivo descomprimido: ../0 Dataset/steam_games.json


## **📂Procesamiento del 1er archivo: `steam_games.json.gz`**


- Tomamos los datos del archivo JSON, los transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [8]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.

row = []
with open("../0 Dataset/steam_games.json", "r", encoding="utf-8") as archivo:
    for linea in archivo:
        try:
            objeto_json = json.loads(linea)
            row.append(objeto_json)
        except json.JSONDecodeError:
            print(f"Error de formato JSON en la línea: {linea}")

#Convertir la lista de objetos JSON en un DataFrame
df_steam_games = pd.DataFrame(row)
#Hacemos una copia del dataset sucio para explorarlo en el EDA
df_steam_games_sucio = pd.DataFrame(row)
# Veamos unos registros al azar
df_steam_games.sample(2)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
96769,NaN,NaN,Comic Book: The Movie,Comic Book: The Movie,http://store.steampowered.com/app/540490/Comic...,2011-11-01,"[Movie, Comedy]",http://steamcommunity.com/app/540490/reviews/?...,[Captions available],2.99,False,540490,NaN
11154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Usando la funcion personalizada `data_type_check` invocada desde `data_utils.py` vemos las variables categóricas, numéricas, dimensiones del dataframe, nulos, tipos de datos e informacion acerca de los datos faltantes o nulos de cada columna    


In [15]:
data_type_check(df_steam_games)


 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (120445, 13)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0      publisher       20.00    80.00        96362    object
1         genres       23.95    76.05        91593    object
2       app_name       26.68    73.32        88312    object
3          title       24.98    75.02        90360    object
4            url       26.68    73.32        88310    object
5   release_date       24.96    75.04        90377    object
6           tags       26.54    73.46        88473    object
7    reviews_url       26.68    73.32        88312    object
8          specs       26.12    73.88        88980    object
9          price       25.54    74.46        89687    object
10  early_access       26.68    73.32        88310    object
11            id       26.68    73.32        88312    object
12     developer       23.94    76.06        91609    object


Observamos que los tipos de datos no son los que necesitamos, vamos a transformar estos datos en los formatos correctos para su posterior uso.

#### 🔁 **TRANSFORM**
  * ✅ Columnas irrelevantes (que no nos sirven para el problema que queremos resolver)
   developer, id. 
  * ✅ Datos faltantes o nulos: **.dropna**
  * ✅ Registros o filas repetidas: **drop_duplicates**
  * ✅ Variables categoricas: Errores tipográficos

In [16]:
'''
Eliminamos columnas irrelevantes para el problema que queremos resolver
'''
print("Dimensiones previas a limpieza:")
print(df_steam_games.shape)
# Eliminamos columnas que no vamos a usar
df_steam_games.drop(columns=["publisher","title","url","early_access","reviews_url","specs"], inplace=True)

'''
Transformacion app_name, tags, developer a string.
'''
#Transformamos en lote las columnas que no necesitan tratamiento especial
df_steam_games = df_steam_games.astype({
    'app_name': 'string',
    'tags': 'string', 
    'developer': 'string',
    'id': 'string'
})


'''
# Revisamos cómo queda nuestro dataframe después del drop
'''
print("Dimensiones post limpieza:")
print(df_steam_games.shape)

data_type_check(df_steam_games)


Dimensiones previas a limpieza:
(120445, 13)
Dimensiones post limpieza:
(120445, 7)

 Resumen del dataframe 'df_steam_games': 

Dimensiones:  (120445, 7)
        columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0        genres       23.95    76.05        91593          object
1      app_name       26.68    73.32        88312  string[python]
2  release_date       24.96    75.04        90377          object
3          tags       26.54    73.46        88473  string[python]
4         price       25.54    74.46        89687          object
5            id       26.68    73.32        88312  string[python]
6     developer       23.94    76.06        91609  string[python]


#### **ID**
Eliminamos repetidos y nulos

In [17]:
# Eliminar duplicados en la columna 'id'
df_steam_games.drop_duplicates(subset='id', inplace=True)
# Eliminar filas con valores nulos en la columna 'id'
df_steam_games.dropna(subset=['id'], inplace=True)
#Terminamos reseteando los indices
df_steam_games.reset_index(drop=True, inplace=True)

#### **Developer**

In [18]:
df_steam_games['developer'].fillna('Dato Faltante', inplace=True)

#### **release_date** 
- Cambiamos tipo de dato a int, 
- Se extrae el año de la fecha en la columna 'release_date' 
- Reemplazamos los datos sin fecha por -1

In [19]:
print("Así se ven las fechas antes de la conversión")
print(df_steam_games['release_date'].head(2))  

# Convertir release_date a datetime (opcional, si es necesario para análisis futuro)
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

# Extraer el año y sobrescribir la columna release_date con el año extraído
df_steam_games['release_date'] = df_steam_games['release_date'].dt.year.fillna(-1).astype(int)  
# Reemplazar NaN con -1 y convertir a entero

# Imprimir resultado
print("Así se ven las fechas después de la conversión (solo el año)")
print(df_steam_games['release_date'].head(2))


Así se ven las fechas antes de la conversión
0    2018-01-04
1    2018-01-04
Name: release_date, dtype: object
Así se ven las fechas después de la conversión (solo el año)
0    2018
1    2018
Name: release_date, dtype: int32


#### **price**
- Asignamos valores correspondientes a precios con textos adicional.
- Convierte la columna 'price' a tipo float.
- Rellena los valores nulos resultantes con -1.

In [20]:
print("Los nulos en price son:") 
df_steam_games['price'].isnull().sum()

Los nulos en price son:


1377

Visualizamos los valores no numericos en precios a solucionar:

In [21]:
unique_prices = df_steam_games['price'].unique()
print(unique_prices)

[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 nan
 'Free' 10.99 1.59 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99
 5.99 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 'Install Now'
 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 64.99
 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5 'Play Now'
 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.39
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0
 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0
 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2.89 'Play the Demo' 99.0
 87.94 599.0 8.98 9.69 0.49 9.98 9.95 7.0 'Starting at $499.00'
 'Starting at $449.00' 12.8

In [22]:
# Limpiar valores no numéricos
non_numeric_values = ['Free', 'Free to Play', 'Free Demo', 'Play for Free!', 'Free Mod', 'Install Now', 'Play Now', 'Third-party', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Starting at', 'Free to Try', 'Free Movie', 'Free to Use', 'Play the Demo']
df_steam_games['price'] = df_steam_games['price'].apply(lambda x: 0 if x in non_numeric_values else x)

# Convertir la columna 'price' a tipo string para limpiar valores con texto adicional
df_steam_games['price'] = df_steam_games['price'].astype(str)

# Limpiar valores con texto adicional y convertir la columna 'price' a tipo float
df_steam_games['price'] = df_steam_games['price'].replace(to_replace=r'[^0-9.]', value='', regex=True)
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')

# Print the first two rows to see the cleaned prices
print(df_steam_games['price'].head(3))


0    4.99
1     NaN
2    0.00
Name: price, dtype: float64


Asignamos el valor 0 si el juego tiene precio faltante y en sus tags se encuentra que el juego es gratis.

In [23]:
# Lista de palabras clave para buscar en las etiquetas
palabras_clave = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Free Mod', 'Free to Try', 'Free Movie', 'Free to Use', 'Play the Demo' ]

# Verificar si el precio del juego es nulo
if df_steam_games['price'].isnull().any():
    # Verificar si alguna de las palabras clave está en las etiquetas para cada fila y si falta el precio
    df_steam_games['precio_faltante'] = df_steam_games['price'].isna() & df_steam_games['tags'].apply(lambda x: any(palabra in x for palabra in palabras_clave if isinstance(x, str)))

    # Reemplazar los valores de precio faltantes con 0 si la fila correspondiente tiene alguna palabra clave en las etiquetas
    df_steam_games.loc[df_steam_games['precio_faltante'], 'price'] = 0

    # Eliminar la columna temporal
    df_steam_games.drop('precio_faltante', axis=1, inplace=True)

# Imprimir las primeras tres filas para confirmar los cambios
print(df_steam_games['price'].head(3))

# Verificar si hay valores nulos en la columna 'price'
if df_steam_games['price'].isnull().any():
    # Asignar -1 a los valores nulos en la columna 'price'
    df_steam_games['price'].fillna(-1, inplace=True)

# Imprimir los nulos en la columna 'price'
print("Los nulos en price son:") 
print(df_steam_games['price'].isnull().sum())

0    4.99
1    0.00
2    0.00
Name: price, dtype: float64
Los nulos en price son:
0


#### **genres**:

- Rellenamos la informacion faltante de genre, con los genre disponibles en la columna tags


In [24]:
print("Resumen previo")
data_type_check(df_steam_games[['genres']])
# Conjunto de géneros a copiar
generos_a_copiar = {'Simulation', 'Adventure', 'Strategy', 'Education', 'Photo Editing', 'Massively Multiplayer', 'Accounting', 'Video Production', 'Design & Illustration', 'Racing', 'Web Publishing', 'Utilities', 'Software Training', 'Sports', 'Action', 'Indie', 'Audio Production', 'Animation & Modeling', 'Casual', 'RPG'}

# Función para rellenar genres desde tags
def rellenar_genres(tags, genres):
    try:
        # Convertir las etiquetas (tags) a una lista
        tags_list = ast.literal_eval(tags)
        # Filtrar los géneros a copiar
        nuevos_generos = [genre for genre in generos_a_copiar if genre in tags_list]        
        # Eliminar 'Free to Play' y 'Early Access'
        nuevos_generos = [genre for genre in nuevos_generos if genre not in ['Free to Play', 'Early Access']]        
        # Si no hay nuevos géneros, devolver los originales
        if not nuevos_generos:
            return genres        
        # Devolver los nuevos géneros como lista
        return nuevos_generos
    
    except (ValueError, SyntaxError):
        # Manejar errores de evaluación
        pass
    
    # Si hay algún error, o la evaluación no es una lista, devolver los géneros originales como lista
    return genres

# Aplicar la función para rellenar genres desde tags
df_steam_games['genres'] = df_steam_games.apply(lambda row: rellenar_genres(row['tags'], row['genres']), axis=1)
# Eliminar 'Free to Play' y 'Early Access'
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
#Observamos la cantidad de nulos
print(" ")
print("Resumen luego de el relleno de datos")
df_info = data_type_check(df_steam_games[['genres']])

Resumen previo


IndexError: list index out of range

#### **📤 LOAD**

In [ ]:
# Donde vamos a guardar
LIMPIO = '../0 Dataset/steam_games_LIMPIO.csv'
SUCIO = '../0 Dataset/steam_games_SUCIO.csv'

#Guardamos los cambios en CSV
df_steam_games.to_csv(LIMPIO, index=False)
df_steam_games_sucio.to_csv(SUCIO, index=False)

# Leemos ambos archivo CSV en un DataFrame de pandas
steam_games = pd.read_csv(LIMPIO)
df_steam_games_sucio = pd.read_csv(SUCIO)

# Convertimos los DataFrame de pandas a una tabla de PyArrow
table_steam_games = pa.Table.from_pandas(steam_games)
table_df_steam_games_sucio = pa.Table.from_pandas(df_steam_games_sucio)

# Escribimos las tabla en un archivo Parquet
pq.write_table(table_steam_games, '../0 Dataset/steam_games_LISTO.parquet')
pq.write_table(table_df_steam_games_sucio, '../0 Dataset/steam_games_SUCIO.parquet')

# Borramos los csv ya que nos sirve más en parquet
os.remove(LIMPIO)
os.remove(SUCIO)

# Dejamos información de muestra acerca de los archivos
data_type_check(steam_games)


 Resumen del Dataframe 
Dimensiones:  (32132, 7)
        columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0        genres       97.07     2.93          942    object
1      app_name      100.00     0.00            1    object
2  release_date      100.00     0.00            0     int64
3          tags       99.50     0.50          162    object
4         price      100.00     0.00            0   float64
5            id      100.00     0.00            0     int64
6     developer      100.00     0.00            0    object
